# Test PyTorch Setup (Module 2 — Deep Learning & NLP)

Goal: quickly verify the environment is ready for the Module 2 projects in this repo.


## What this notebook checks
- Python & OS info
- PyTorch installation and version
- GPU/accelerator availability (CUDA / MPS)
- A tiny forward pass to verify basic functionality
- Reproducibility seed


In [1]:
# If you see "No module named torch", install requirements first:
# pip install -r requirements.txt
import os, sys, platform, time
import torch
print('Python:', sys.version.split()[0])
print('Platform:', platform.platform())
print('PyTorch:', torch.__version__)


Python: 3.13.9
Platform: Windows-11-10.0.26200-SP0
PyTorch: 2.9.1+cpu


In [2]:
def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    # Apple Silicon
    if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')

device = get_device()
print('Selected device:', device)
if device.type == 'cuda':
    print('CUDA device:', torch.cuda.get_device_name(0))
    print('CUDA capability:', torch.cuda.get_device_capability(0))
    print('CUDA version (runtime):', torch.version.cuda)
    print('cuDNN enabled:', torch.backends.cudnn.enabled)


Selected device: cpu


In [3]:
def seed_everything(seed: int = 42):
    import random
    import numpy as np
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)
print('Seed set ✅')


Seed set ✅


## Quick sanity checks

These checks confirm tensors work on the selected device and that a small model can run forward & backward.


In [4]:
import torch
x = torch.randn(3, 4, device=device)
w = torch.randn(4, 2, device=device)
y = x @ w
print('x shape:', x.shape)
print('y shape:', y.shape)
print('y sample:', y[0].tolist())


x shape: torch.Size([3, 4])
y shape: torch.Size([3, 2])
y sample: [0.38002967834472656, -0.42683613300323486]


In [5]:
# Tiny model forward/backward
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(10, 32),
    nn.ReLU(),
    nn.Linear(32, 2)
).to(device)

batch = torch.randn(16, 10, device=device)
targets = torch.randint(0, 2, (16,), device=device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

optimizer.zero_grad(set_to_none=True)
logits = model(batch)
loss = criterion(logits, targets)
loss.backward()
optimizer.step()

print('Forward/backward OK ✅')
print('Loss:', float(loss))


Forward/backward OK ✅
Loss: 0.7203152179718018


C:\Users\ORENS\AppData\Local\Temp\ipykernel_32052\499383295.py:23: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:837.)
  print('Loss:', float(loss))


## (Optional) DataLoader check

This is useful if you plan to use datasets for CV/NLP.


In [6]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(torch.randn(128, 10), torch.randint(0, 2, (128,)))
loader = DataLoader(dataset, batch_size=32, shuffle=True)

xb, yb = next(iter(loader))
print('Batch shapes:', xb.shape, yb.shape)
print('First labels:', yb[:8].tolist())


Batch shapes: torch.Size([32, 10]) torch.Size([32])
First labels: [1, 0, 0, 1, 1, 0, 0, 0]


## Notes
- If device is CPU only: it's still OK for learning and small experiments.
- If CUDA is available but training is slow: run the performance notebook to benchmark.
